# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [9]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking'].to_numpy()

# Display a sample of y
y[0:5]

array([0, 0, 0, 1, 0])

In [10]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis = 1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [11]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [12]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [13]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
len(X.columns)

11

In [14]:
# Define the number of hidden nodes for the first hidden layer
hidden_node_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_node_layer2 = 3

# Define the number of neurons in the output layer
output = 1

In [15]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units = hidden_node_layer1, activation = 'relu', input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units = hidden_node_layer2, activation= 'relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units = output, activation = 'sigmoid'))


In [16]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [17]:
# Compile the Sequential model
nn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [21]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5145 - accuracy: 0.7540
Epoch 2/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5131 - accuracy: 0.7573
Epoch 3/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5116 - accuracy: 0.7581
Epoch 4/100
38/38 [==============================] - 0s 4ms/step - loss: 0.5100 - accuracy: 0.7598
Epoch 5/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5087 - accuracy: 0.7640
Epoch 6/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5074 - accuracy: 0.7656
Epoch 7/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5062 - accuracy: 0.7656
Epoch 8/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5045 - accuracy: 0.7690
Epoch 9/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5035 - accuracy: 0.7673
Epoch 10/100
38/38 [==============================] - 0s 3ms/step - loss: 0.5018 - accuracy: 0.7715
Epoch 11/

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [22]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5330 - accuracy: 0.7375 - 45ms/epoch - 3ms/step
Loss: 0.532950222492218, Accuracy: 0.737500011920929


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [24]:
# Set the model's file path
from google.colab import drive
drive.mount('/content/drive')
# file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
# nn_model.save(file_path)

Mounted at /content/drive


In [26]:
model_path = '/content/drive/My Drive/student_loans.keras'  # Specify the path and model name
nn_model.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [28]:
# Set the model's file path - the model's path was set above in prior step. Using same naming here for efficiency.


# Load the model to a new object
nn_imported = tf.keras.models.load_model(model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [35]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled)

# Display a sample of the predictions
predictions[0:10]

13/13 [==============================] - 0s 3ms/step


array([[0.16810338],
       [0.33844146],
       [0.8692992 ],
       [0.5513257 ],
       [0.87887096],
       [0.3774072 ],
       [0.87887096],
       [0.018106  ],
       [0.6199793 ],
       [0.4010506 ]], dtype=float32)

In [36]:
# Save the predictions to a DataFrame and round the predictions to binary results
pred_df = pd.DataFrame(predictions, columns = ["predictions"]).round(0)
pred_df.head(10)

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
5,0.0
6,1.0
7,0.0
8,1.0
9,0.0


### Step 4: Display a classification report with the y test data and predictions

In [37]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, pred_df))

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       188
           1       0.75      0.76      0.76       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

It is not entirely clear what the "credit_ranking" column represents. I am assuming that it represents a likelihood that the individual will repay their loan, with 0 being less likely and 1 being more likely. This is not clarified anywhere in the instructions or the data. Additionally, I am interpreting that the student loan options the model would recommend is referring to different types of loans and interest rates that would be recommended to the borrower (as opposed to recommending whether or not the borrower should be approved for a loan). I'm not very clear on what this question is asking, because in this exercise we built the model based on data provided to us in a starter file. So, are you asking us what data was in the starter file? Or what additional, separate data we would need to collect? Or what data we would need in order to derive the data provided in the starter file? If I assume the question is asking what data needs to be obtained from the borrower in order to derive the data points in the model, then the following data should be collected:

1. Borrower's existing loan statements/history (payment history)
2. Home address (location parameter)
3. Borrower's transcript provides school name (alumni success), student's GPA (GPA ranking), study major/degree (stem degree score, study major), time to completion
4. Financial aid information (financial aid score)

I would like to point out that the data points I listed above are not necessarily indicators of whether or not someone will repay their student loans or what type of loan they should be offered. They are just the data points provided in the starter file for the purposes of the exercise. Better indicators of someone's ability to pay back a refinanced loan would be current income, expenses, and location to determine cost of living.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

The model would use content-based filtering, making recommendations by comparing the contents of a user profile, which includes the data collected above. This allows for relevant and specialized recommendations based on the borrower's specific financial situation.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

The data needed for this model includes a lot of personal information and could be a challenge to invite users to provide all of the information. Would be important to have security measures in place to be able to protect and keep private the data provided.

An additional challenge of this model is that past behavior does not predict future results.  Basing recommendations solely on a borrower's past metrics and behavior is not a 100% guarantee of whether or not they will repay a loan. And relying solely on metrics like that can have a consequence of denying people access to financial resources and perpetuation existing societal inequalities and injustices.

